In [1]:
import tabula
import pandas as pd
import numpy as np
import webreg_to_cal
import re

In [2]:
dfs = tabula.read_pdf("../../Downloads/webregMainFA23.pdf", pages="all")

In [3]:
df = dfs[0]

In [4]:
alltext = df.columns[0]; alltext

'Carter Joseph Tran - Fall Quarter 2023\rSubjectSectionGradeStatus /\rTitleTypeInstructorUnitsDaysTimeBLDGRoomAction\rCourseCodeOption(Position)\rCAT 1Culture, Art, and Technology 1A00LEAlgaze, GuillermoL4.00MWF9:00a-9:50aFAH1301Enrolled\rA03DIM11:00a-11:50aHSS2305B\rTiefenbruck, Janine\rDSC 10Principles of Data ScienceA00LEL4.00MWF10:00a-10:50aMANDEB-210Enrolled\rLoBue\rA01DIM8:00p-8:50pMANDEB-210\rLab\rLAW3:00p-3:50pPCYNH109\rFinal ExamFISa 12/09/20237:00p-9:59pTBATBA\rMATH 31AHHonors Linear AlgebraA00LERabin, Jeffrey MarkL4.00MWF3:00p-3:50pPETER103Enrolled\rA01DITh9:00a-9:50aHSS4025\rMidtermMITh 10/26/20238:00p-9:20pPETER103\rMidtermMITu 11/28/20238:00p-9:20pPETER103\rFinal ExamFIF 12/15/20233:00p-5:59pTBATBA'

In [5]:
df = df.iloc[:-2,:-2]

In [6]:
headers = df.iloc[0].values
df.columns = headers
df.drop(index=0, axis=0, inplace=True)

In [7]:
def fix_first_letter(string):
    try:
        index = alltext.index(string) - 1
        return alltext[index] + string
    except Exception:
        return string
# df["Title"].apply(fix_first_letter)
# df["Instructor"].apply(fix_first_letter)

In [8]:
df = df.assign(Title = df["Title"].apply(fix_first_letter), Instructor = df["Instructor"].apply(fix_first_letter))

In [9]:
df = df.rename(columns={"Subject\rCourse": "Subject", "Section\rCode": "Section", "Grade\rOption": "Grade", "Status /\r(Position)": "Status"})
df.reset_index(drop=True, inplace=True)
df

,Subject,Title,Section,Type,Instructor,Grade,Units,Days,Time,BLDG,Room,Status
0,CAT 1,"Culture, Art, and Technology 1",A00,LE,"Algaze, Guillermo",L,4.00,MWF,9:00a-9:50a,FAH,1301,Enrolled
1,NaN,NaN,A03,DI,NaN,NaN,NaN,M,11:00a-11:50a,HSS,2305B,NaN
2,DSC 10,Principles of Data Science,A00,LE,"iefenbruck, Janine\roBue",L,4.00,MWF,10:00a-10:50a,MANDE,B-210,Enrolled
3,NaN,NaN,A01,DI,NaN,NaN,NaN,M,8:00p-8:50p,MANDE,B-210,NaN
4,NaN,\rLab,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,LA,NaN,NaN,NaN,W,3:00p-3:50p,PCYNH,109,NaN
6,NaN,Final Exam,NaN,FI,NaN,NaN,NaN,Sa 12/09/2023,7:00p-9:59p,TBA,TBA,NaN
7,MATH 31AH,Honors Linear Algebra,A00,LE,"Rabin, Jeffrey Mark",L,4.00,MWF,3:00p-3:50p,PETER,103,Enrolled
8,NaN,NaN,A01,DI,NaN,NaN,NaN,Th,9:00a-9:50a,HSS,4025,NaN
9,NaN,Midterm,NaN,MI,NaN,NaN,NaN,Th 10/26/2023,8:00p-9:20p,PETER,103,NaN


In [10]:
for i, row in df.iterrows():
    if row["Subject"] is np.nan:
        df.iloc[i]["Subject"] = df.iloc[i-1]["Subject"]
        df.iloc[i]["Title"] = df.iloc[i-1]["Title"]
        df.iloc[i]["Instructor"] = df.iloc[i-1]["Instructor"]
        df.iloc[i]["Grade"] = df.iloc[i-1]["Grade"]
        df.iloc[i]["Units"] = df.iloc[i-1]["Units"]
    if row["Status"] is np.nan:
        df.iloc[i]["Status"] = df.iloc[i-1]["Status"]

In [11]:
df

,Subject,Title,Section,Type,Instructor,Grade,Units,Days,Time,BLDG,Room,Status
0,CAT 1,"Culture, Art, and Technology 1",A00,LE,"Algaze, Guillermo",L,4.00,MWF,9:00a-9:50a,FAH,1301,Enrolled
1,CAT 1,"Culture, Art, and Technology 1",A03,DI,"Algaze, Guillermo",L,4.00,M,11:00a-11:50a,HSS,2305B,Enrolled
2,DSC 10,Principles of Data Science,A00,LE,"iefenbruck, Janine\roBue",L,4.00,MWF,10:00a-10:50a,MANDE,B-210,Enrolled
3,DSC 10,Principles of Data Science,A01,DI,"iefenbruck, Janine\roBue",L,4.00,M,8:00p-8:50p,MANDE,B-210,Enrolled
4,DSC 10,Principles of Data Science,NaN,NaN,"iefenbruck, Janine\roBue",L,4.00,NaN,NaN,NaN,NaN,Enrolled
5,DSC 10,Principles of Data Science,NaN,LA,"iefenbruck, Janine\roBue",L,4.00,W,3:00p-3:50p,PCYNH,109,Enrolled
6,DSC 10,Principles of Data Science,NaN,FI,"iefenbruck, Janine\roBue",L,4.00,Sa 12/09/2023,7:00p-9:59p,TBA,TBA,Enrolled
7,MATH 31AH,Honors Linear Algebra,A00,LE,"Rabin, Jeffrey Mark",L,4.00,MWF,3:00p-3:50p,PETER,103,Enrolled
8,MATH 31AH,Honors Linear Algebra,A01,DI,"Rabin, Jeffrey Mark",L,4.00,Th,9:00a-9:50a,HSS,4025,Enrolled
9,MATH 31AH,Honors Linear Algebra,NaN,MI,"Rabin, Jeffrey Mark",L,4.00,Th 10/26/2023,8:00p-9:20p,PETER,103,Enrolled


In [12]:
match = re.search(r'\b\w+\b Quarter \b\d{4}\b', alltext)
quarter = match.group(0) if match else "Fall 2023"
quarter

'Fall Quarter 2023'